In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os
from pandas import json_normalize
from scipy import stats
from scipy.signal import savgol_filter

In [2]:
df = pd.read_csv("C:/Users/user/Desktop/GA4_project/preproccessed_df.csv")

# 시간 단위 컬럼으로 변환
df['utc_time'] = pd.to_datetime(df['utc_time'])

df.head(3)

C:\Users\user\AppData\Local\Temp\ipykernel_12932\477848715.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/user/Desktop/GA4_project/preproccessed_df.csv")


,fullVisitorId,visitStartTime,channelGrouping,continent,subContinent,country,city,visits,pageviews,hits,...,bounces,transactionRevenue,campaign,source,medium,keyword,browser,operatingSystem,deviceCategory,utc_time
0,1131660440785968503,1472830385,Organic Search,Asia,Western Asia,Turkey,Izmir,1,1,1,...,1,0,(not set),google,organic,(not provided),Chrome,Windows,desktop,2016-09-02 15:33:05+00:00
1,377306020877927890,1472880147,Organic Search,Oceania,Australasia,Australia,not available in demo dataset,1,1,1,...,1,0,(not set),google,organic,(not provided),Firefox,Macintosh,desktop,2016-09-03 05:22:27+00:00
2,3895546263509774583,1472865386,Organic Search,Europe,Southern Europe,Spain,Madrid,1,1,1,...,1,0,(not set),google,organic,(not provided),Chrome,Windows,desktop,2016-09-03 01:16:26+00:00


In [3]:
df = df[df['country'] == 'United States']

df.head(3)

,fullVisitorId,visitStartTime,channelGrouping,continent,subContinent,country,city,visits,pageviews,hits,...,bounces,transactionRevenue,campaign,source,medium,keyword,browser,operatingSystem,deviceCategory,utc_time
67,3746051970600816343,1472843059,Referral,Americas,Northern America,United States,Mountain View,1,1,1,...,1,0,(not set),sites.google.com,referral,(not provided),Chrome,Linux,desktop,2016-09-02 19:04:19+00:00
96,2486407044795377188,1472830426,Referral,Americas,Northern America,United States,not available in demo dataset,1,1,1,...,1,0,(not set),siliconvalley.about.com,referral,(not provided),Chrome,Windows,desktop,2016-09-02 15:33:46+00:00
102,2870162892603227525,1472858877,Paid Search,Americas,Northern America,United States,not available in demo dataset,1,1,1,...,1,0,AW - Dynamic Search Ads Whole Site,google,cpc,6qEhsCssdK0z36ri,Chrome,Android,mobile,2016-09-02 23:27:57+00:00


In [4]:
# 켐페인 컬럼에 어떤 값들이 있는지 확인
df['campaign'].unique()

array(['(not set)', 'AW - Dynamic Search Ads Whole Site',
       'Data Share Promo', 'test-liyuhz', 'AW - Accessories',
       'Retail (DO NOT EDIT owners nophakun and tianyu)', 'AW - Apparel',
       'AW - Electronics', 'All Products', 'Data Share'], dtype=object)

In [5]:
# transaction revenue 컬럼이 0이면 0, 0을 초과하면 1로 표시되는 새로운 컬럼 추가(revenue_occurred)
df['revenue_occurred'] = np.where(df['transactionRevenue'] == 0, 0, 1)

df.head(3)

,fullVisitorId,visitStartTime,channelGrouping,continent,subContinent,country,city,visits,pageviews,hits,...,transactionRevenue,campaign,source,medium,keyword,browser,operatingSystem,deviceCategory,utc_time,revenue_occurred
67,3746051970600816343,1472843059,Referral,Americas,Northern America,United States,Mountain View,1,1,1,...,0,(not set),sites.google.com,referral,(not provided),Chrome,Linux,desktop,2016-09-02 19:04:19+00:00,0
96,2486407044795377188,1472830426,Referral,Americas,Northern America,United States,not available in demo dataset,1,1,1,...,0,(not set),siliconvalley.about.com,referral,(not provided),Chrome,Windows,desktop,2016-09-02 15:33:46+00:00,0
102,2870162892603227525,1472858877,Paid Search,Americas,Northern America,United States,not available in demo dataset,1,1,1,...,0,AW - Dynamic Search Ads Whole Site,google,cpc,6qEhsCssdK0z36ri,Chrome,Android,mobile,2016-09-02 23:27:57+00:00,0


In [10]:
# 각 켐페인별로 총 세션 수 계산
total_inflow = df['campaign'].value_counts()

# 각 켐페인별로 purchase_sessions 계산
purchase_sessions = df.groupby('campaign')['revenue_occurred'].sum()

# revenue_rate 계산 및 데이터 프레임 생성
revenue_rate = (purchase_sessions / total_inflow) * 100
campaign_df = pd.DataFrame({
    'total_inflow': total_inflow,
    'purchase_sessions': purchase_sessions,
    'revenue_rate(%)': revenue_rate.round(2)
})

# 결과 확인
campaign_df

,total_inflow,purchase_sessions,revenue_rate(%)
campaign,,,
(not set),340437,10493,3.08
AW - Accessories,6622,128,1.93
AW - Apparel,44,1,2.27
AW - Dynamic Search Ads Whole Site,13780,320,2.32
AW - Electronics,77,0,0.00
All Products,4,0,0.00
Data Share,1,0,0.00
Data Share Promo,3638,9,0.25
Retail (DO NOT EDIT owners nophakun and tianyu),50,1,2.00
